In [ ]:
using Revise, Plots
using BifurcationKit
const BK = BifurcationKit
using LaTeXStrings
using DifferentialEquations
using Setfield

default(gridlinewidth = 2, gridalpha= 0.05, linewidth = 1.5, framestyle = :box, size = (210, 200), 
    xtickfont=font(9), ytickfont=font(9), legendfont=font(8), guidefont=font(12))

In [ ]:
# vector field of the problem
function RHIZOIDS(Y, p)
	(;alpha0,alpha,beta,gamma0,gamma,dm,dr,df,kr,kf,kc,nr,nf,D) = p
	m1,R1,F1,m2,R2,F2,diff = Y
    out = similar(Y)
	out[1] = alpha0 + alpha*(R1^nr)/(kr^nr + R1^nr) - dm*m1 - kc*m1*F1
	out[2] = beta*m1 - dr*R1
	out[3] = gamma0 + gamma*(R1^nf)/(kf^nf + R1^nf) - df*F1 + D*(F2 - F1)
	out[4] = alpha0 + alpha*(R2^nr)/(kr^nr + R2^nr) - dm*m2 - kc*m2*F2
	out[5] = beta*m2 - dr*R2
	out[6] = gamma0 + gamma*(R2^nf)/(kf^nf + R2^nf) - df*F2 + D*(F1 - F2)
    out[7] = abs(R1 - R2) - diff
    out
end

function rhizoids(dz,z,p,t)
	(;alpha0,alpha,beta,gamma0,gamma,dm,dr,df,kr,kf,kc,nr,nf,D) = p
    m1,R1,F1,m2,R2,F2,diff = z[1],z[2],z[3],z[4],z[5],z[6],z[7]
	dz[1] = alpha0 + alpha*(R1^nr)/(kr^nr + R1^nr) - dm*m1 - kc*m1*F1
	dz[2] = beta*m1 - dr*R1
	dz[3] = gamma0 + gamma*(R1^nf)/(kf^nf + R1^nf) - df*F1 + D*(F2 - F1)
	dz[4] = alpha0 + alpha*(R2^nr)/(kr^nr + R2^nr) - dm*m2 - kc*m2*F2
	dz[5] = beta*m2 - dr*R2
	dz[6] = gamma0 + gamma*(R2^nf)/(kf^nf + R2^nf) - df*F2 + D*(F1 - F2)
    dz[7] = abs(R1 - R2) - diff

end


#0 = alpha0 + alpha*(R1^nr)/(kr^nr + R1^nr) - dm*(dr/beta)*R1 - kc*(dr/beta)*R1*((D + df)*(gamma0 + gamma*(R1^nf)/(kf^nf + R1^nf)) + D*(gamma0 + gamma*(R2^nf)/(kf^nf + R2^nf)))/(df*df - 2*D*df)

# parameters used in the model
par_com = (alpha0 = 0.4, gamma0 = 0.1, beta = 0.1, alpha = 2., gamma = 0.78, kc = 1.,
           kr = 1.2, kf = 2., nr = 4., nf = 4., dm = 1., dr = 0.1, df = 1., D = 0.5)

# oscillations
#par_com = (alpha0 = 0.2, gamma0 = 0.1, beta = 1., alpha = 6., gamma = 5, kc = 1.,
#           kr = 1.2, kf = 2., nr = 4., nf = 4., dm = 1., dr = 0.1, df = 0.02, D = 0.0)


recordCO(x, p; kwargs...) = (m1 = x[1], R1 = x[2], F1 = x[3], m2 = x[4], R2 = x[5], F2 = x[6], diff = x[7])

# initial condition
z1 = [0.6,0.6,0.3,0.7,0.7,0.3,0]
z2 = [1.5,1.5,0.6,1.4,1.4,0.5,0]
#z3 = [0.1,1.1,1.1,5.1,1.3,0.1,2]

tfinal = 600.
sol1 = DifferentialEquations.solve(ODEProblem(rhizoids,z1,(0.,tfinal),par_com),Rosenbrock23())
sol2 = DifferentialEquations.solve(ODEProblem(rhizoids,z2,(0.,tfinal),par_com),Rosenbrock23())
#sol3 = DifferentialEquations.solve(ODEProblem(rhizoids,z3,(0.,tfinal),par_com),Rosenbrock23())

plot(sol1,vars=(2), xlims = (0.,tfinal), ylims = (0.,3.), zlims = (0.0,2.), linecolor = :red, linewidth = 2, label = L"$R_1$", layout = 1, size = (600,300))
plot!(sol1,vars=(5), xlims = (0.,tfinal), ylims = (0.,3.), zlims = (0.0,2.), linecolor = :green, linewidth = 2, label = L"$R_2$")
#plot!(sol1,vars=(7), xlims = (0.,tfinal), ylims = (0.,3.), zlims = (0.0,2.), linecolor = :black, inewidth = 2, label = L"|R_1 - R_2|")

plot!(sol2,vars=(2), xlims = (0.,tfinal), ylims = (0.,3.), zlims = (0.0,2.), linecolor = :red, linestyle = :dash, linewidth = 2, label = L"$R_1$", layout = 1, size = (600,300))
plot!(sol2,vars=(5), xlims = (0.,tfinal), ylims = (0.,3.), zlims = (0.0,2.), linecolor = :green, linestyle = :dash, linewidth = 2, label = L"$R_2$")
#plot!(sol2,vars=(7), xlims = (0.,tfinal), ylims = (0.,3.), zlims = (0.0,2.), linecolor = :black, linestyle = :dash, linewidth = 2, label = L"|R_1 - R_2|")


In [ ]:
# Bifurcation Problem
zb1 = [0.4,0.4,0.4,0.4,0.4,0.4,0]
zb2 = [1.4,1.4,0.4,1.3,1.3,0.3,0]
zb3 = [1.1,1.1,0.4,0.7,0.7,0.4,2]
zb4 = [1.1,1.1,0.4,0.4,0.4,0.4,2]
zb5 = [1.1,1.1,0.4,0.4,0.4,0.4,2]

zb1 = sol1[end]
zb2 = sol2[end]
#zb3 = sol3[end]

pmax = 1.;

par_com1= (alpha0 = 0., gamma0 = 0.1, beta = 0.1, alpha = 2., gamma = 0.78, kc = 1.,
           kr = 1.2, kf = 2., nr = 4., nf = 4., dm = 1., dr = 0.1, df = 1., D = 0.6)

par_com2 = (alpha0 = 0.5, gamma0 = 0.1, beta = 0.1, alpha = 2., gamma = 0.78, kc = 1.,
           kr = 1.2, kf = 2., nr = 4., nf = 4., dm = 1., dr = 0.1, df = 1., D = 0.6)

par_com3 = (alpha0 = 0.5, gamma0 = 0.1, beta = 0.1, alpha = 2., gamma = 0.78, kc = 1.,
           kr = 1.2, kf = 2., nr = 4., nf = 4., dm = 1., dr = 0.1, df = 1., D = 0.6)

par_com4 = (alpha0 = 0., gamma0 = 0.1, beta = 0.1, alpha = 2., gamma = 0.78, kc = 1.,
           kr = 1.2, kf = 2., nr = 4., nf = 4., dm = 1., dr = 0.1, df = 1., D = 0.6)

par_com5 = (alpha0 = 0., gamma0 = 0.1, beta = 0.1, alpha = 2., gamma = 0.78, kc = 1.,
           kr = 1.2, kf = 2., nr = 4., nf = 4., dm = 1., dr = 0.1, df = 1., D = 0.6)

prob1 = BifurcationProblem(RHIZOIDS, zb1, par_com1, IndexLens(1); record_from_solution = recordCO)
prob2 = BifurcationProblem(RHIZOIDS, zb2, par_com2, IndexLens(1); record_from_solution = recordCO)
prob3 = BifurcationProblem(RHIZOIDS, zb3, par_com3, IndexLens(1); record_from_solution = recordCO)
prob4 = BifurcationProblem(RHIZOIDS, zb2, par_com4, IndexLens(1); record_from_solution = recordCO)
prob5 = BifurcationProblem(RHIZOIDS, zb2, par_com5, IndexLens(1); record_from_solution = recordCO)


# continuation parameters
opts_br1 = ContinuationPar(p_min = 0.0, p_max = pmax, dsmin = 0.001, ds = 0.001, dsmax = 0.001, 
    max_steps = 10000)

opts_br2 = ContinuationPar(p_min = 0.0, p_max = pmax, dsmin = 0.0001, ds = 0.0001, dsmax = 0.0001, 
    max_steps = 75000)

opts_br3 = ContinuationPar(p_min = 0.0, p_max = pmax, dsmin = 0.00001, ds = 0.00001, dsmax = 0.00001,
    max_steps = 250000)

opts_br4 = ContinuationPar(p_min = 0.0, p_max = pmax, dsmin = 0.001, ds = 0.001, dsmax = 0.005, 
    max_steps = 10000)

opts_br5 = ContinuationPar(p_min = 0.0, p_max = pmax, dsmin = 0.001, ds = 0.001, dsmax = 0.005, 
    max_steps = 10000)


# compute the branch of solutions
br1 = continuation(prob1, PALC(), opts_br1; normC = norminf)
br2 = continuation(prob2, PALC(), opts_br2; normC = norminf)
br3 = continuation(prob3, PALC(), opts_br3; normC = norminf)
br4 = continuation(prob4, PALC(), opts_br4; normC = norminf)
br5 = continuation(prob5, PALC(), opts_br5; normC = norminf)


In [ ]:
# plot the branch

plt = plot(br1, xlims = (0.,1), ylims = (0.,2), linecolors = :green, linewidthstable = 2,
     linewidthunstable = 1., grid = false, background_color=:transparent, 
     xlabel = L"$\alpha_0$", ylabel = L"$R$", xaxis=:lin, yaxis=:lin, label = false)

plot!(br3,br2, xlabel = L"$\alpha_0$", ylabel = L"$R$",
     linecolors = :black, markercolors = :red, 
     linewidthstable = 2, linewidthunstable = 1, label = false)

display(plt)

savefig("rhizoids_bifurcation_2cells.pdf")

In [ ]:
pmax = 8.

sn_codim2_1 = continuation(br1, 2,  IndexLens(5),
	ContinuationPar(opts_br1, p_min = 0., p_max = pmax, dsmin = 1e-6, ds = 0.0001, dsmax = 0.002, max_steps = 2000);
	normC = norminf,
	# compute both sides of the initial condition
	bothside = true,
	# detection of codim 2 bifurcations
	detect_codim2_bifurcation = 2,
	)

sn_codim2_2 = continuation(br3, 3, IndexLens(5),
	ContinuationPar(opts_br1, p_min = 0., p_max = pmax, dsmin = 1e-6, ds = 0.0001, dsmax = 0.002, max_steps = 3500);
	normC = norminf,
	# compute both sides of the initial condition
	bothside = true,
	# detection of codim 2 bifurcations
	detect_codim2_bifurcation = 2,
	)

sn_codim2_3 = continuation(br3, 6, IndexLens(5),
	ContinuationPar(opts_br1, p_min = 0., p_max = pmax, dsmin = 1e-6, ds = 0.0001, dsmax = 0.002, max_steps = 6300);
	normC = norminf,
	# compute both sides of the initial condition
	bothside = true,
	# detection of codim 2 bifurcations
	detect_codim2_bifurcation = 2,
	)

sn_codim2_4 = continuation(br3, 4, IndexLens(5),
	ContinuationPar(opts_br1, p_min = 0., p_max = pmax, dsmin = 1e-6, ds = 0.0001, dsmax = 0.002, max_steps = 3500);
	normC = norminf,
	# compute both sides of the initial condition
	bothside = true,
	# detection of codim 2 bifurcations
	detect_codim2_bifurcation = 2,
	)


In [ ]:
#plot2 = plot(sn_codim2_1, vars = (:alpha0, :R1), markercolor = :red)
# plot!(sn_codim2_2, vars = (:alpha0, :R1))
# plot!(sn_codim2_3, vars = (:alpha0, :R1))
# plot!(sn_codim2_4, vars = (:alpha0, :R1))
#plot!(plot2, br1,br5, xlims=(0.3,0.7), linecolors = :green, markercolor = :red, xlabel = L"$\alpha_0$", ylabel = L"$R$")
#display(plot2)

cod2 = plot(sn_codim2_1, vars = IndexLens(1,5),  markersize = 2, markercolor = :red, color = :black, alpha = 1)
plot!(sn_codim2_3, vars = IndexLens(1,5),  markersize = 2, markercolor = :red, color = :green, alpha = 1)
plot!(sn_codim2_4, vars = IndexLens(1,5),  markersize = 2, markercolor = :red, color = :green, alpha = 1)
plot!(sn_codim2_1, vars = IndexLens(1,5),  markersize = 2, markercolor = :red, color = :black, alpha = 1)

plot!(cod2, xlims=(0.,6), ylims=(0.,1),legend=false, grid = false, color = :black,
     xlabel = L"$\gamma$", ylabel = L"$\alpha_0$", background_color=:transparent, size = (215,210))
display(cod2)

savefig("bif_cod2_diffusion_2cells.pdf")